In [1]:
# og dataset link: https://data.sfgov.org/Public-Safety/Traffic-Crashes-Resulting-in-Injury/ubvf-ztfx/about_data

In [32]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [33]:
og_df = pd.read_csv("../src/data/Traffic_Crashes_Resulting_in_Injury_20240518.csv")

/var/folders/j1/4b075fzj2xdbn0txbrmsq7hm0000gn/T/ipykernel_40201/2720964843.py:1: DtypeWarning: Columns (3,15) have mixed types. Specify dtype option on import or set low_memory=False.
  og_df = pd.read_csv("../src/data/Traffic_Crashes_Resulting_in_Injury_20240518.csv")


working with weather

In [34]:
# display
og_df[['weather_1','weather_2', 'unique_id']].groupby(['weather_1','weather_2'], dropna=False).count()

unique_id
weather_1                weather_2                             
Clear                    Cloudy                             160
                         Fog                                  6
                         Fog / Visibility                     1
                         Fog / Visibility: 10 ft              1
                         Fog / Visibility: 500 ft             1
                         Not Stated                       47414
                         Other                               29
                         Other: NOT ON SCENE                  1
                         Other: Night                         1
                         Other: SUN GLARING                   1
                         Other: SUN IN WB VIEW                1
                         Other: SUNNY                         1
                         Other: SUNRISE                       1
                         Other: SUNSET                        1
                         Other: Sunny                         1
                         Other: Sunset                        1
                         Other: smoke cloud                   1
                         Raining                             43
                         Snowing                              3
                         Wind                               108
                         NaN                                686
Cloudy                   Fog                                 38
                         Fog / Visibility                     7
                         Fog / Visibility: 100 ft             1
                         Fog / Visibility: 15 ft              1
                         Fog / Visibility: 30 ft              1
                         Not Stated                        5123
                         Other                               28
                         Other: Drizzling                     1
                         Other: HEAVY MIST                    1
                         Other: LIGHT MIST                    1
                         Other: Light drizzle (rain)          1
                         Other: MIST                          2
                         Other: MISTING                       1
                         Other: Mist                          1
                         Other: Misty                         1
                         Other: Overcast                      1
                         Other: SLIGHT MIST                   1
                         Other: SMOKEY                        1
                         Other: SMOKEY SKIES                  1
                         Raining                            515
                         Snowing                              1
                         Wind                                32
                         NaN                                 67
Fog                      Not Stated                         243
                         Other                                4
                         Other: FALLING ASH                   1
                         Other: WET PAVEMENT                  1
                         Other: drizzle                       1
                         Wind                                 1
                         NaN                                  3
Fog / Visibility         Not Stated                           1
Fog / Visibility: 10 ft  Not Stated                           1
Fog / Visibility: 800 ft Not Stated                           1
Not Stated               Not Stated                         638
                         NaN                                 13
Other                    Not Stated                         222
                         NaN                                  4
Other: MISTING           Not Stated                           1
Other: NOT AT SCENE      Not Stated                           1
Other: NOT ON SCENE      Not Stated                           1
Other: Unknown        

In [48]:
nan_list = ['Not Stated', 'Other: NOT ON SCENE', 'Other: NOT AT SCENE', 'Other: Unknown', np.NaN]
sun_list = [r'^Other: [sS][uU][nN].*']
fog_list = [r'^Fog.*']
mist_list = [r'Other:.*[mM][iI][sS][tT].*'] # I'm going to assume drizzling is not mist. or rain.
nan_toofew_list = ['Other: Night', 'Other: smoke cloud', 
                   'Other: Drizzling', 'Other: Light drizzle (rain)',
                   'Other: Overcast', 'Other: SMOKEY', 'Other: SMOKEY SKIES',
                   'Other: FALLING ASH', 'Other: WET PAVEMENT', 'Other: drizzle']

In [49]:
rep_lists = [nan_list, sun_list, fog_list, mist_list, nan_toofew_list]
rep_withs = ['None', 'Sun', 'Fog', 'Mist', 'Other']
regexes = [False, True, True, True, False]

In [50]:
for i in range(len(rep_lists)):    
    og_df['weather_1'] = og_df['weather_1'].replace(to_replace = rep_lists[i], 
                                                    value = rep_withs[i],
                                                    regex = regexes[i])
    og_df['weather_2'] = og_df['weather_2'].replace(to_replace = rep_lists[i], 
                                                    value = rep_withs[i],
                                                    regex = regexes[i])

In [51]:
og_df[['weather_1','weather_2', 'unique_id']].groupby(['weather_1','weather_2'], dropna=False).count()
# note the vast majority (48k) happens in clear weather, no other modifiers.

unique_id
weather_1 weather_2           
Clear     Cloudy           160
          Fog                9
          None           48101
          Other             31
          Raining           43
          Snowing            3
          Sun                7
          Wind             108
Cloudy    Fog               48
          Mist               8
          None            5190
          Other             33
          Raining          515
          Snowing            1
          Wind              32
Fog       None             249
          Other              7
          Wind               1
Mist      None               1
None      None             654
Other     None             226
Raining   Fog                9
          None            3441
          Other              7
          Wind              76
Snowing   None               2
Wind      None              47

In [52]:
og_df['weather'] = og_df.apply(lambda x: [x['weather_1'], x['weather_2']], axis = 1)

In [53]:
from sklearn.preprocessing import MultiLabelBinarizer

In [54]:
mlb = MultiLabelBinarizer()

In [55]:
mlb.fit_transform(og_df['weather'])

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [ ]:
# next steps: encode them

In [56]:
from sklearn.compose import ColumnTransformer

In [62]:
ct = ColumnTransformer([('test', mlb, ['weather'])])

In [63]:
ct.fit_transform(og_df)

TypeError: MultiLabelBinarizer.fit_transform() takes 2 positional arguments but 3 were given

# below is scratch

## WHAT MIXED TYPES
- case id pkey
- juris

random eda stuff:

In [101]:
og_df['collision_time']

0        17:45:00
1        14:27:00
2        22:51:00
3        22:18:00
4        13:35:00
           ...   
59004    14:10:00
59005    14:00:00
59006    21:40:00
59007    20:11:00
59008    15:25:00
Name: collision_time, Length: 59009, dtype: object

In [93]:
og_df[og_df['collision_time'].isna()]

,unique_id,cnn_intrsctn_fkey,cnn_sgmt_fkey,case_id_pkey,tb_latitude,tb_longitude,geocode_source,geocode_location,collision_datetime,collision_date,...,party2_move_pre_acc,point,data_as_of,data_updated_at,data_loaded_at,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods
340,19666,26050000.0,4801101.0,130603293,37.773900,-122.437623,SFPD-CROSSROADS,CITY STREET,07/22/2013 12:00:00 AM,2013 July 22,...,Proceeding Straight,POINT (-122.437622909 37.77390032),07/22/2013 12:00:00 AM,04/26/2023 12:00:00 AM,04/26/2024 01:04:18 PM,24.0,24.0,4.0,11.0,9.0
399,35479,26751000.0,NaN,17027320,37.799843,-122.436046,SFPD-CROSSROADS,CITY STREET,02/10/2017 12:00:00 AM,2017 February 10,...,Not Stated,POINT (-122.436046174 37.799842953),02/10/2017 12:00:00 AM,04/26/2023 12:00:00 AM,04/26/2024 01:04:18 PM,17.0,17.0,4.0,6.0,13.0
570,45394,32859000.0,184101.0,141101024,NaN,NaN,SFPD-CROSSROADS,CITY STREET,12/31/2014 12:00:00 AM,2014 December 31,...,Making U Turn,NaN,12/31/2014 12:00:00 AM,04/26/2023 12:00:00 AM,04/26/2024 01:04:18 PM,NaN,NaN,NaN,NaN,NaN
577,8889,23756000.0,NaN,130037222,37.759675,-122.402080,SFPD-CROSSROADS,CITY STREET,01/14/2013 12:00:00 AM,2013 January 14,...,Not Stated,POINT (-122.402079752 37.759675432),01/14/2013 12:00:00 AM,04/26/2023 12:00:00 AM,04/26/2024 01:04:18 PM,54.0,54.0,2.0,9.0,26.0
762,42150,27210000.0,NaN,5430820,37.774167,-122.459486,SFPD-CROSSROADS,CITY STREET,11/05/2011 12:00:00 AM,2011 November 05,...,Not Stated,POINT (-122.459486118 37.774166934),11/05/2011 12:00:00 AM,04/26/2023 12:00:00 AM,04/26/2024 01:04:18 PM,5.0,5.0,8.0,4.0,12.0
798,74780,24132000.0,12024000.0,230474726,37.761984,-122.417181,SFPD-INTERIM DB,CITY STREET,07/07/2023 12:00:00 AM,2023 July 07,...,Proceeding Straight,POINT (-122.417180507 37.761983953),04/19/2024 12:00:00 AM,04/19/2024 12:00:00 AM,04/26/2024 01:04:18 PM,53.0,53.0,3.0,2.0,20.0
902,9289,23677000.0,NaN,140691131,37.765189,-122.395807,SFPD-CROSSROADS,CITY STREET,08/18/2014 12:00:00 AM,2014 August 18,...,Backing,POINT (-122.395806777 37.765188633),08/18/2014 12:00:00 AM,04/26/2023 12:00:00 AM,04/26/2024 01:04:18 PM,54.0,54.0,1.0,9.0,26.0
2000,73937,20618000.0,NaN,240061250,37.729211,-122.400834,SFPD-INTERIM DB,CITY STREET,01/28/2024 12:00:00 AM,2024 January 28,...,Proceeding Straight,POINT (-122.400834015 37.729211438),04/16/2024 12:00:00 AM,04/16/2024 12:00:00 AM,04/26/2024 01:04:18 PM,87.0,87.0,2.0,9.0,1.0
4251,49329,26622000.0,NaN,190820233,37.784531,-122.438020,SFPD-INTERIM DB,CITY STREET,10/30/2019 12:00:00 AM,2019 October 30,...,Proceeding Straight,POINT (-122.438019672 37.784530919),11/20/2019 12:00:00 AM,04/26/2023 12:00:00 AM,04/26/2024 01:04:18 PM,103.0,103.0,4.0,11.0,15.0
6393,23124,24592000.0,170000.0,140389267,37.783014,-122.397963,SFPD-CROSSROADS,CITY STREET,05/09/2014 12:00:00 AM,2014 May 09,...,Stopped In Road,POINT (-122.397963237 37.783014077),05/09/2014 12:00:00 AM,04/26/2023 12:00:00 AM,04/26/2024 01:04:18 PM,32.0,32.0,1.0,10.0,34.0


In [54]:
# check nulls in each column:
og_df.apply(lambda x: sum(x.isnull()), axis = 0)

unique_id                           0
cnn_intrsctn_fkey                   7
cnn_sgmt_fkey                   32565
case_id_pkey                        0
tb_latitude                       154
tb_longitude                      154
geocode_source                      0
geocode_location                    0
collision_datetime                  0
collision_date                      0
collision_time                     59
accident_year                       0
month                               0
day_of_week                         8
time_cat                           52
juris                               0
officer_id                       1081
reporting_district               6510
beat_number                      4546
primary_rd                          0
secondary_rd                      106
distance                           79
direction                           1
weather_1                           0
weather_2                         796
collision_severity                  0
type_of_coll

In [26]:
og_df[og_df['weather_2'] == 'Other: FALLING ASH']['collision_severity']

44221    Injury (Complaint of Pain)
Name: collision_severity, dtype: object

In [27]:
og_df['type_of_collision'].unique()

array(['Rear End', 'Sideswipe', 'Vehicle/Pedestrian', 'Hit Object',
       'Other', 'Head-On', 'Overturned', 'Broadside', 'Not Stated'],
      dtype=object)

In [89]:
og_df[['unique_id','dph_col_grp_description']].groupby('dph_col_grp_description').count()

,unique_id
dph_col_grp_description,
Bicycle Only,1061
Bicycle-Parked Car,425
Bicycle-Pedestrian,459
Bicycle-Unknown/Not Stated,3
Pedestrian Only or Pedestrian-Parked Car,31
Unknown/Not Stated,14
Vehicle(s) Only Involved,34715
Vehicle-Bicycle,7959
Vehicle-Bicycle-Pedestrian,18


In [76]:
og_df['dph_col_grp_description'].unique()

array(['Vehicle(s) Only Involved', 'Vehicle-Bicycle',
       'Vehicle-Pedestrian', 'Bicycle Only', 'Bicycle-Pedestrian',
       'Pedestrian Only or Pedestrian-Parked Car', 'Bicycle-Parked Car',
       'Unknown/Not Stated', 'Bicycle-Unknown/Not Stated',
       'Vehicle-Bicycle-Pedestrian'], dtype=object)

In [97]:
sum(og_df['lighting'] == 'Not Stated')

592

In [16]:
# check number of unique values in each column:
og_df.apply(lambda x: len(x.unique()), axis = 0)

unique_id                       59009
cnn_intrsctn_fkey                6045
cnn_sgmt_fkey                    7282
case_id_pkey                    59009
tb_latitude                     35053
tb_longitude                    33654
geocode_source                      3
geocode_location                    1
collision_datetime              58508
collision_date                   7022
collision_time                   1441
accident_year                      22
month                              12
day_of_week                         8
time_cat                            7
juris                              14
officer_id                       3937
reporting_district                886
beat_number                      3462
primary_rd                       1216
secondary_rd                     1681
distance                          863
direction                           7
weather_1                          15
weather_2                          39
collision_severity                  5
type_of_coll

In [99]:
og_df['Neighborhoods']

0         81.0
1          8.0
2         32.0
3         10.0
4        100.0
         ...  
59004     79.0
59005     97.0
59006     40.0
59007     53.0
59008     20.0
Name: Neighborhoods, Length: 59009, dtype: float64

In [100]:
og_df['SF Find Neighborhoods']

0         81.0
1          8.0
2         32.0
3         10.0
4        100.0
         ...  
59004     79.0
59005     97.0
59006     40.0
59007     53.0
59008     20.0
Name: SF Find Neighborhoods, Length: 59009, dtype: float64

In [92]:
sum(og_df['party1_type'] == 'Bicycle')

1